In [2]:
# importing the required libaries
import os
import pickle
import mediapipe as mp
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
mp_hands=mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)

Data_dir='./data1'
i=0
data=[]
labels=[]
for dir_ in os.listdir(Data_dir)[9:12]:
    for img_path in os.listdir(os.path.join(Data_dir,dir_)):
        data_aux=[]
        img = cv2.imread(os.path.join(Data_dir,dir_,img_path))
        img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        results=hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x=hand_landmarks.landmark[i].x
                    y=hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)

            data.append(data_aux)
            labels.append(dir_)
f=open('data.pickle','wb')
pickle.dump({'data':data,'labels':labels},f)
f.close()

In [4]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

with open('data.pickle', 'rb') as file:
    data_dict = pickle.load(file)

lengths = [len(sublist) for sublist in data_dict['data']]
unique_lengths = set(lengths)

if len(unique_lengths) == 1:
    data_array = np.array(data_dict['data'])
else:
    max_length = max(lengths)
    data_array = np.array([sublist + [0] * (max_length - len(sublist)) for sublist in data_dict['data']])

labels_array = np.array(data_dict['labels'])

target_array = np.zeros((len(data_array), data_array.shape[1]))
target_array[:data_array.shape[0], :data_array.shape[1]] = data_array

x_train, x_test, y_train, y_test = train_test_split(target_array, labels_array, test_size=0.2, shuffle=True, stratify=labels_array)

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_predict = model.predict(x_test)
accuracy = accuracy_score(y_predict, y_test)
print(accuracy * 100)

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()


100.0


In [5]:
import cv2
import mediapipe as mp
import pickle
import numpy as np


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Loading the model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Starting real time video
cap = cv2.VideoCapture(0)

while True:
    data_aux1 = []
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  
                hand_landmarks,  
                mp_hands.HAND_CONNECTIONS,  
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
            for landmark in hand_landmarks.landmark:
                data_aux1.append(landmark.x)
                data_aux1.append(landmark.y)

        
        if len(data_aux1) < 84:
            data_aux1.extend([0] * (84 - len(data_aux1)))

        
        if len(data_aux1) == 84:
            prediction = model.predict([np.asarray(data_aux1)])
            print("Prediction:", prediction)
        else:
            print(f"Error: Expected 84 features, but got {len(data_aux1)} features.")

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Prediction: ['R']
Prediction: ['R']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['J']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['S']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['R']
Prediction: ['J']
Prediction: ['R']
Prediction: ['U']
Prediction

KeyboardInterrupt: 